# Hierarchical Clustering

**Hierarchical clustering** refers to a class of clustering methods that seek to build a **hierarchy** of clusters, in which some clusters contain others. In this assignment, we will explore a top-down approach, recursively bipartitioning the data using k-means.

In [1]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.cluster import KMeans                # we'll be using scikit-learn's KMeans for this assignment
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize
%matplotlib inline

In [2]:
!ls

CLU06-NB01.ipynb
DCphrd4eEemzYQ60aJAwVA_fc87f6033fa34a2ab644e47ef86539cc_em_utilities.py.zip
EZW10-I8EemJfgqR2HI2sA_97cfb8979d704bf299ddc0b287ba68a6_CLU06-NB01.ipynb.zip
_167fc84b78dc145609e919983b475aaa_people_wiki.csv.zip
_395a4cfb2299d1655f1ef6bf6cc4f71b_people_wiki_tf_idf.npz.zip
_96eadbec4d43a0b0870dde27d0652fb2_people_wiki_map_index_to_word.json.zip
closing-annotated.pdf
em_utilities.py
i5zBDt4bEemhxxJUtZcQoA_7fb43aab245846f09a8c48977883f0b6_people_wiki.sframe.zip
people_wiki.csv
people_wiki.sframe
people_wiki_map_index_to_word.json
people_wiki_tf_idf.npz
week6_hierarchical_clustering_assignment_1.ipynb


In [3]:
wiki = pd.read_csv('people_wiki.csv')

As in the previous assignment, we extract the TF-IDF vector of each document.

For your convenience, we extracted the TF-IDF vectors from the dataset. The vectors are packaged in a sparse matrix, where the i-th row gives the TF-IDF vectors for the i-th document. Each column corresponds to a unique word appearing in the dataset.

To load in the TF-IDF vectors, run

In [4]:
def load_sparse_csr(filename):
    loader = np.load(filename)
    data = loader['data']
    indices = loader['indices']
    indptr = loader['indptr']
    shape = loader['shape']
    
    return csr_matrix( (data, indices, indptr), shape)

tf_idf = load_sparse_csr('people_wiki_tf_idf.npz')


In [5]:
import json

with open('people_wiki_map_index_to_word.json', 'r') as filehandle:
    map_index_to_word = json.load(filehandle)

To be consistent with the k-means assignment, let's normalize all vectors to have unit norm.

In [6]:
tf_idf = normalize(tf_idf)

## Bipartition the Wikipedia dataset using k-means

Recall our workflow for clustering text data with k-means:

1. Load the dataframe containing a dataset, such as the Wikipedia text dataset.
2. Extract the data matrix from the dataframe.
3. Run k-means on the data matrix with some value of k.
4. Visualize the clustering results using the centroids, cluster assignments, and the original dataframe. We keep the original dataframe around because the data matrix does not keep auxiliary information (in the case of the text dataset, the title of each article).

Let us modify the workflow to perform bipartitioning:

1. Load the dataframe containing a dataset, such as the Wikipedia text dataset.
2. Extract the data matrix from the dataframe.
3. Run k-means on the data matrix with k=2.
4. Divide the data matrix into two parts using the cluster assignments.
5. Divide the dataframe into two parts, again using the cluster assignments. This step is necessary to allow for visualization.
6. Visualize the bipartition of data.

We'd like to be able to repeat Steps 3-6 multiple times to produce a **hierarchy** of clusters such as the following:
```
                      (root)
                         |
            +------------+-------------+
            |                          |
         Cluster                    Cluster
     +------+-----+             +------+-----+
     |            |             |            |
   Cluster     Cluster       Cluster      Cluster
```
Each **parent cluster** is bipartitioned to produce two **child clusters**. At the very top is the **root cluster**, which consists of the entire dataset.

Now we write a wrapper function to bipartition a given cluster using k-means. There are three variables that together comprise the cluster:

* `dataframe`: a subset of the original dataframe that correspond to member rows of the cluster
* `matrix`: same set of rows, stored in sparse matrix format
* `centroid`: the centroid of the cluster (not applicable for the root cluster)

Rather than passing around the three variables separately, we package them into a Python dictionary. The wrapper function takes a single dictionary (representing a parent cluster) and returns two dictionaries (representing the child clusters).

In [14]:
def bipartition(cluster, maxiter=400, num_runs=4, seed=None):
    '''cluster: should be a dictionary containing the following keys
                * dataframe: original dataframe
                * matrix:    same data, in matrix format
                * centroid:  centroid for this particular cluster'''
    
    data_matrix = cluster['matrix']
    dataframe   = cluster['dataframe']
    
    # Run k-means on the data matrix with k=2. We use scikit-learn here to simplify workflow.
    kmeans_model = KMeans(n_clusters=2, max_iter=maxiter, n_init=num_runs, random_state=seed, n_jobs=1)    
    kmeans_model.fit(data_matrix)
    centroids, cluster_assignment = kmeans_model.cluster_centers_, kmeans_model.labels_
    
    # Divide the data matrix into two parts using the cluster assignments.
    data_matrix_left_child, data_matrix_right_child = data_matrix[cluster_assignment==0], \
                                                      data_matrix[cluster_assignment==1]
    
    # Divide the dataframe into two parts, again using the cluster assignments.
    dataframe['cluster_assignment'] = cluster_assignment # minor format conversion
    dataframe_left_child, dataframe_right_child     = dataframe[dataframe['cluster_assignment']==0], \
                                                      dataframe[dataframe['cluster_assignment']==1]
        
    
    # Package relevant variables for the child clusters
    cluster_left_child  = {'matrix': data_matrix_left_child,
                           'dataframe': dataframe_left_child,
                           'centroid': centroids[0]}
    cluster_right_child = {'matrix': data_matrix_right_child,
                           'dataframe': dataframe_right_child,
                           'centroid': centroids[1]}
    
    return (cluster_left_child, cluster_right_child)

The following cell performs bipartitioning of the Wikipedia dataset. Allow 2+ minutes to finish.

Note. For the purpose of the assignment, we set an explicit seed (`seed=1`) to produce identical outputs for every run. In pratical applications, you might want to use different random seeds for all runs.

In [ ]:
## Takes long time, do not run

wiki_data = {'matrix': tf_idf, 'dataframe': wiki} # no 'centroid' for the root cluster
left_child, right_child = bipartition(wiki_data, maxiter=100, num_runs=6, seed=1)

KeyboardInterrupt: 

In [9]:
left_child

{'centroid': array([4.72706878e-06, 0.00000000e+00, 4.53832211e-06, ...,
        1.10898073e-04, 9.02386276e-05, 2.11503833e-05]),
 'dataframe':                                                      URI  \
 1           <http://dbpedia.org/resource/Alfred_J._Lewy>   
 3      <http://dbpedia.org/resource/Franz_Rottensteiner>   
 5            <http://dbpedia.org/resource/Sam_Henderson>   
 7          <http://dbpedia.org/resource/Trevor_Ferguson>   
 9             <http://dbpedia.org/resource/Cathy_Caruth>   
 ...                                                  ...   
 59061             <http://dbpedia.org/resource/Rod_Wilt>   
 59062  <http://dbpedia.org/resource/Scott_Baker_(judge)>   
 59063  <http://dbpedia.org/resource/Dragoljub_Ojdani%...   
 59064            <http://dbpedia.org/resource/Oz_Bengur>   
 59070         <http://dbpedia.org/resource/Fawaz_Damrah>   
 
                          name  \
 1              Alfred J. Lewy   
 3         Franz Rottensteiner   
 5               Sam

In [10]:
right_child

{'centroid': array([0.00000000e+00, 3.42736698e-06, 0.00000000e+00, ...,
        1.20952547e-04, 7.96999832e-05, 2.04635492e-05]),
 'dataframe':                                                      URI  \
 0            <http://dbpedia.org/resource/Digby_Morrell>   
 2            <http://dbpedia.org/resource/Harpdog_Brown>   
 4                   <http://dbpedia.org/resource/G-Enka>   
 6            <http://dbpedia.org/resource/Aaron_LaCrate>   
 8             <http://dbpedia.org/resource/Grant_Nelson>   
 ...                                                  ...   
 59065  <http://dbpedia.org/resource/Dee_Brown_(basket...   
 59066           <http://dbpedia.org/resource/Olari_Elts>   
 59067       <http://dbpedia.org/resource/Scott_F._Crago>   
 59068  <http://dbpedia.org/resource/David_Cass_(footb...   
 59069          <http://dbpedia.org/resource/Keith_Elias>   
 
                                     name  \
 0                          Digby Morrell   
 2                          Harp

## Visualize the bipartition

We provide you with a modified version of the visualization function from the k-means assignment. For each cluster, we print the top 5 words with highest TF-IDF weights in the centroid and display excerpts for the 8 nearest neighbors of the centroid.

In [11]:
def display_single_tf_idf_cluster(cluster, map_index_to_word):
    '''map_index_to_word: SFrame specifying the mapping betweeen words and column indices'''
    
    wiki_subset   = cluster['dataframe']
    tf_idf_subset = cluster['matrix']
    centroid      = cluster['centroid']
    
    # Print top 5 words with largest TF-IDF weights in the cluster
    idx = centroid.argsort()[::-1]
    for i in range(5):
        print('{0}:{1:.3f}'.format({v:k for k, v in map_index_to_word.items()}[idx[i]], centroid[idx[i]])),
    print('')
    
    # Compute distances from the centroid to all data points in the cluster.
    distances = pairwise_distances(tf_idf_subset, [centroid], metric='euclidean').flatten()
    # compute nearest neighbors of the centroid within the cluster.
    nearest_neighbors = distances.argsort()
    # For 8 nearest neighbors, print the title as well as first 180 characters of text.
    # Wrap the text at 80-character mark.
    for i in range(8):
        text = ' '.join(wiki_subset.iloc[nearest_neighbors[i]]['text'].split(None, 25)[0:25])
        print('* {0:50s} {1:.5f}\n  {2:s}\n  {3:s}'.format(wiki_subset.iloc[nearest_neighbors[i]]['name'],
              distances[nearest_neighbors[i]], text[:90], text[90:180] if len(text) > 90 else ''))
    print('')

In [12]:
map_index_to_word['category']

546431

Let's visualize the two child clusters:

In [71]:
display_single_tf_idf_cluster(left_child, map_index_to_word)

she:0.022
university:0.015
her:0.013
he:0.012
served:0.010

* Kayee Griffin                                      0.97357
  kayee frances griffin born 6 february 1950 is an australian politician and former australi
  an labor party member of the new south wales legislative council serving
* %C3%81ine Hyland                                   0.97368
  ine hyland ne donlon is emeritus professor of education and former vicepresident of univer
  sity college cork ireland she was born in 1942 in athboy co
* Christine Robertson                                0.97372
  christine mary robertson born 5 october 1948 is an australian politician and former austra
  lian labor party member of the new south wales legislative council serving
* Anita Kunz                                         0.97468
  anita e kunz oc born 1956 is a canadianborn artist and illustratorkunz has lived in london
   new york and toronto contributing to magazines and working
* Barry Sullivan (lawyer)                       

In [73]:
display_single_tf_idf_cluster(right_child, map_index_to_word)

she:0.023
music:0.017
her:0.017
league:0.016
season:0.016

* Patricia Scott                                     0.97144
  patricia scott pat born july 14 1929 is a former pitcher who played in the allamerican gir
  ls professional baseball league for parts of four seasons
* Madonna (entertainer)                              0.97185
  madonna louise ciccone tkoni born august 16 1958 is an american singer songwriter actress 
  and businesswoman she achieved popularity by pushing the boundaries of lyrical
* Janet Jackson                                      0.97261
  janet damita jo jackson born may 16 1966 is an american singer songwriter and actress know
  n for a series of sonically innovative socially conscious and
* Natashia Williams                                  0.97346
  natashia williamsblach born august 2 1978 is an american actress and former wonderbra camp
  aign model who is perhaps best known for her role as shane
* Todd Williams                                      0.9738

The right cluster consists of athletes and artists (singers and actors/actresses), whereas the left cluster consists of non-athletes and non-artists. So far, we have a single-level hierarchy consisting of two clusters, as follows:

```
                                           Wikipedia
                                               +
                                               |
                    +--------------------------+--------------------+
                    |                                               |
                    +                                               +
         Non-athletes/artists                                Athletes/artists
```

Is this hierarchy good enough? **When building a hierarchy of clusters, we must keep our particular application in mind.** For instance, we might want to build a **directory** for Wikipedia articles. A good directory would let you quickly narrow down your search to a small set of related articles. The categories of athletes and non-athletes are too general to facilitate efficient search. For this reason, we decide to build another level into our hierarchy of clusters with the goal of getting more specific cluster structure at the lower level. To that end, we subdivide both the `athletes/artists` and `non-athletes/artists` clusters.